## First model - labeled CT data

In [37]:
#!pip install opencv-python

import pandas as pd
import numpy as np
import cv2
import os
import matplotlib.pyplot as plt

import tensorflow

from tensorflow.keras.utils import to_categorical

from sklearn.model_selection import train_test_split

from tensorflow.keras import callbacks

from joblib import dump, load

In [2]:
def read_ct_img_bydir(target_dir):
    img=cv2.imdecode(np.fromfile(target_dir,dtype=np.uint8),cv2.IMREAD_GRAYSCALE)
    #img = corp_margin(img)
    img=cv2.resize(img,(200,200))
    
    return img

In [3]:
def get_data():
    '''Loads raw data from image names of each class folder and assigns a label to it.'''
    
    target_dir1='../raw_data/labeled_CT_data/NiCT/'
    target_dir2='../raw_data/labeled_CT_data/pCT/'
    target_dir3='../raw_data/labeled_CT_data/nCT/'
    
#     target_dir1='../raw_data/labeled_CT_data/test_CT/'
#     target_dir2='../raw_data/labeled_CT_data/test_CT_2/'
#     target_dir3='../raw_data/labeled_CT_data/test_CT_3/'

    target_list1=[target_dir1+file for file in os.listdir(target_dir1)]
    target_list2=[target_dir2+file for file in os.listdir(target_dir2)]
    target_list3=[target_dir3+file for file in os.listdir(target_dir3)]

    target_list=target_list1+target_list2+target_list3

    # Assign labels: 0: 'non informative CT'
    #                1: 'positive Covid-19 CT'
    #                2: 'negative Covid-19 CT'
    y_list=to_categorical(np.concatenate(np.array([[0]*len(target_list1),
                                               [1]*len(target_list2),
                                               [2]*len(target_list3)])),3)

    X=np.array([read_ct_img_bydir(file) for file in target_list])[:,:,:,np.newaxis]

    return X, y_list

In [4]:
X,y = get_data()

In [5]:
X.shape

(17093, 200, 200, 1)

In [6]:
y.shape

(17093, 3)

In [7]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [8]:
from tensorflow.keras.models import Sequential
from tensorflow.keras import layers

model = Sequential()
model.add(layers.Conv2D(64, (3,3), input_shape=(200, 200, 1), activation="relu"))
model.add(layers.MaxPool2D(pool_size=(2,2))) 

model.add(layers.Conv2D(128, (3,3), activation="relu"))
model.add(layers.MaxPool2D(pool_size=(2,2)))

model.add(layers.Conv2D(64, (3,3), activation="relu"))
model.add(layers.MaxPool2D(pool_size=(2,2))) 

model.add(layers.Conv2D(32, (3,3), activation="relu"))
model.add(layers.MaxPool2D(pool_size=(2,2))) 
          
model.add(layers.Flatten())
model.add(layers.Dense(3, activation="softmax"))

# The compilation
model.compile(loss='categorical_crossentropy', 
              optimizer='adam', metrics = 'accuracy')  # Recommanded optimizer for RNN


In [9]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 198, 198, 64)      640       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 99, 99, 64)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 97, 97, 128)       73856     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 48, 48, 128)       0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 46, 46, 64)        73792     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 23, 23, 64)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 21, 21, 32)        1

In [11]:
# The fit

es = callbacks.EarlyStopping(patience=20, restore_best_weights=True)

history = model.fit(X_train, y_train,
                    batch_size=32,
                    epochs=2,
                    validation_split=0.3,
                    callbacks=[es],
                    verbose=1)


Epoch 1/2
262/262 [==============================] - 847s 3s/step - loss: 0.0913 - accuracy: 0.9685 - val_loss: 0.0457 - val_accuracy: 0.9833
Epoch 2/2
262/262 [==============================] - 949s 4s/step - loss: 0.0352 - accuracy: 0.9888 - val_loss: 0.0381 - val_accuracy: 0.9858


In [ ]:
# The prediction
model.predict(X_test)

In [15]:
pred = model.evaluate(X_test, y_test, verbose=1)

161/161 [==============================] - 101s 628ms/step - loss: 0.0359 - accuracy: 0.9883


In [17]:
pred

[0.035887349396944046, 0.988299548625946]

In [31]:
!pip install pickle5

     |████████████████████████████████| 132 kB 2.0 MB/s eta 0:00:01
  Created wheel for pickle5: filename=pickle5-0.0.11-cp38-cp38-linux_x86_64.whl size=290429 sha256=3dfdd6a6c997dd365dea69c20b9ee58da8183c376caa601c76dab727d279fa0c
  Stored in directory: /home/adduser/.cache/pip/wheels/25/d4/61/dbd8edd1a0d656be7b4267c85db3b61951eb60016a0154a122
Successfully built pickle5
You should consider upgrading via the '/home/adduser/.pyenv/versions/3.8.6/envs/lewagon/bin/python3.8 -m pip install --upgrade pip' command.


In [39]:
dump(model, 'model2.joblib')

TypeError: cannot pickle 'tensorflow.python._tf_stack.StackSummary' object

In [43]:
loaded_model = load('/home/adduser/code/NinLev/Covid_19/notebooks/model2.joblib')

FileNotFoundError: [Errno 2] No such file or directory: '~/code/NinLev/Covid_19/notebooks/model2.joblib'

In [28]:
pwd

'/home/adduser/code/NinLev/Covid_19/notebooks'

In [44]:
#model.fit(X_train, Y_train)
# save the model to disk
filename = 'finalized_model.sav'
pickle.dump(model, open(filename, 'wb'))

# load the model from disk
loaded_model = pickle.load(open(filename, 'rb'))
#result = loaded_model.score(X_test, Y_test)

TypeError: cannot pickle 'weakref' object